In [2]:
import os
import mne
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Subset
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import warnings
import random

warnings.filterwarnings("ignore")

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Global parameters (you can adjust these later)
T = 160
delta = 4
Delta = 8
Gamma = 30
channels = ['Oz..', 'T7..', 'Cz..']


In [3]:
def process_edf(file_path, channels, T=160, delta=4, Delta=8, Gamma=30):
    raw = mne.io.read_raw_edf(file_path, preload=True, verbose=False)
    raw.pick(channels)
    data = raw.get_data().T.astype(np.float32)
    
    # Normalize the data.
    scaler = MinMaxScaler()
    normalized_data = scaler.fit_transform(data).astype(np.float32)
    
    n_samples = normalized_data.shape[0]
    windows = [normalized_data[start:start+T, :] 
               for start in range(0, n_samples - T + 1, delta)]
    windows = np.array(windows, dtype=np.float32)
    
    # Create augmented samples by grouping Gamma consecutive windows with a step of Delta.
    augmented = [windows[i:i+Gamma] 
                 for i in range(0, len(windows) - Gamma + 1, Delta)]
    augmented = np.array(augmented, dtype=np.float32)
    
    return augmented


In [4]:
class PhysionetAugmentedDataset(Dataset):
    def __init__(self, subject_ids, channels, T=160, delta=4, Delta=8, Gamma=30):
        self.subject_ids = subject_ids
        self.channels = channels
        self.T = T
        self.delta = delta
        self.Delta = Delta
        self.Gamma = Gamma
        self.indices = []
        
        for label, subject in enumerate(tqdm(subject_ids, desc="Building index")):
            for r in range(1, 2):
                file_path = f'./files/S{subject:03d}/S{subject:03d}R{r:02d}.edf'
                try:
                    augmented = process_edf(file_path, channels, T, delta, Delta, Gamma)
                except Exception as e:
                    print(f"Error processing {file_path}: {e}")
                    continue
                num_augmented = augmented.shape[0]
                for aug_idx in range(num_augmented):
                    self.indices.append((subject, r, aug_idx, file_path, label))
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        subject, r, aug_idx, file_path, label = self.indices[idx]
        augmented = process_edf(file_path, self.channels, self.T, self.delta, self.Delta, self.Gamma)
        sample = augmented[aug_idx]
        sample_tensor = torch.tensor(sample, dtype=torch.float32)
        return sample_tensor, label
    
subjects = list(range(1, 91))
dataset = PhysionetAugmentedDataset(subjects, channels, T, delta, Delta, Gamma)


Building index: 100%|██████████| 90/90 [00:05<00:00, 17.92it/s]


In [5]:
# Define subject groups:
all_indices = list(range(len(dataset)))
labels = [dataset.indices[i][-1] for i in all_indices]  # label is the last element in the index tuple

# Perform a stratified split (80% train, 20% test)
train_idx, test_idx = train_test_split(all_indices, test_size=0.2, stratify=labels, random_state=42)

# Create subset datasets using the indices.
train_dataset = Subset(dataset, train_idx)
test_dataset = Subset(dataset, test_idx)

# Create DataLoaders for both splits.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=0)


In [11]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(128, 256, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(256, 512, kernel_size=3, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=(1,2))
        self.maxpool2 = nn.MaxPool2d(kernel_size=(2,2))
        self.flatten = nn.Flatten()
        self.dense1 = nn.Linear(153600, 1024)
        # self.dense2 = nn.Linear(1024, 90)
        self.dropout = nn.Dropout2d(0.5) #hyperparameter

    def forward(self, x):
        x = x.permute(0, 3, 1, 2)
        x = F.relu(self.conv1(x))
        x = self.maxpool(x)
        
        x = F.relu(self.conv2(x))
        x = self.maxpool(x)

        x = F.relu(self.conv3(x))
        x = self.maxpool2(x)
        
        x = self.flatten(x)

        x = F.relu(self.dense1(x))
        x = self.dropout(x)
        # x = self.dense2(x)
        return x
    
model = CNN().to(device)

In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

num_epochs = 20  # Adjust as needed

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    all_train_preds = []
    all_train_labels = []

    for batch_data, batch_labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(batch_data)
        loss = criterion(outputs, batch_labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

        preds = outputs.argmax(dim=1)
        all_train_preds.extend(preds.cpu().numpy())
        all_train_labels.extend(batch_labels.cpu().numpy())
    
    train_accuracy = accuracy_score(all_train_labels, all_train_preds)
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Train Accuracy = {train_accuracy:.4f}")

Epoch 1/20: 100%|██████████| 668/668 [05:10<00:00,  2.15it/s]


Epoch 1: Loss = 3.1628, Train Accuracy = 0.2207


Epoch 2/20: 100%|██████████| 668/668 [05:11<00:00,  2.14it/s]


Epoch 2: Loss = 1.9437, Train Accuracy = 0.4610


Epoch 3/20: 100%|██████████| 668/668 [05:13<00:00,  2.13it/s]


...

Epoch 18: Loss = 0.1608, Train Accuracy = 0.9479


Epoch 19/20: 100%|██████████| 668/668 [05:15<00:00,  2.11it/s]


Epoch 19: Loss = 0.1330, Train Accuracy = 0.9550


Epoch 20/20: 100%|██████████| 668/668 [05:16<00:00,  2.11it/s]

Epoch 20: Loss = 0.1204, Train Accuracy = 0.9604


In [11]:
model.eval()
all_test_preds = []
all_test_labels = []
with torch.no_grad():
    for batch_data, batch_labels in tqdm(test_loader, desc="Evaluating"):
        batch_data = batch_data.to(device)
        batch_labels = batch_labels.to(device)
        outputs = model(batch_data)
        preds = outputs.argmax(dim=1)
        all_test_preds.extend(preds.cpu().numpy())
        all_test_labels.extend(batch_labels.cpu().numpy())
    
    test_accuracy = accuracy_score(all_test_labels, all_test_preds)
    print(f"Epoch {epoch+1}: Test Accuracy = {test_accuracy:.4f}")

Evaluating: 100%|██████████| 167/167 [01:15<00:00,  2.22it/s]

Epoch 20: Test Accuracy = 0.9079


In [54]:
class SiameseDataset(Dataset): 
    def __init__(self, base_dataset, similar_prob=1.0, min_time_gap=5):
        self.base_dataset = base_dataset
        self.similar_prob = similar_prob
        self.min_time_gap = min_time_gap

        self.subject_file_to_indices = {}
        for i, entry in enumerate(base_dataset.indices):
            subject = entry[0]
            file_path = entry[3]
            key = (subject, file_path)
            if key not in self.subject_file_to_indices:
                self.subject_file_to_indices[key] = []
            self.subject_file_to_indices[key].append(i)
            
        self.all_indices = list(range(len(base_dataset)))

    def __len__(self):
        return len(self.base_dataset)
    
    def __getitem__(self, idx):
        if random.random() < self.similar_prob:
            pair_label = 1

            index1 = idx
            subject, recording, aug_idx, file_path, label = self.base_dataset.indices[index1]
            # Only choose candidates that come from the same subject and same file.
            key = (subject, file_path)
            candidate_indices = self.subject_file_to_indices.get(key, [])

            index2 = random.choice(candidate_indices)
            diff = abs(self.base_dataset.indices[index1][2] - self.base_dataset.indices[index2][2])

            while index2 == index1 or (diff < self.min_time_gap):
                index2 = random.choice(candidate_indices)
                diff = abs(self.base_dataset.indices[index1][2] - self.base_dataset.indices[index2][2])
           
        else:
            pair_label = 0
            index1 = idx
            subject1 = self.base_dataset.indices[index1][-1]
            index2 = random.choice(self.all_indices)
            
            while self.base_dataset.indices[index2][-1] == subject1:
                index2 = random.choice(self.all_indices)

        sample1, _ = self.base_dataset[index1]
        sample2, _ = self.base_dataset[index2]
        return sample1, sample2, torch.tensor(pair_label, dtype=torch.float32)

In [59]:
siamese_dataset = SiameseDataset(dataset, similar_prob=0.5, min_time_gap=5)
all_indices = list(range(len(siamese_dataset)))
# The label for each sample is the subject stored as the last element in the base_dataset index.
siamese_labels = [siamese_dataset.base_dataset.indices[i][-1] for i in all_indices]

# Perform a stratified split (80% train, 20% test)
siamese_train_idx, siamese_test_idx = train_test_split(
    all_indices, 
    test_size=0.2, 
    stratify=siamese_labels, 
    random_state=42
)

# Create Subset datasets.
siamese_train_dataset = Subset(siamese_dataset, siamese_train_idx)
siamese_test_dataset = Subset(siamese_dataset, siamese_test_idx)

siamese_train_loader = DataLoader(siamese_train_dataset, batch_size=32, shuffle=True, num_workers=0)
siamese_test_loader = DataLoader(siamese_test_dataset, batch_size=32, shuffle=False, num_workers=0)

In [60]:
class Siamese(nn.Module):
    def __init__(self):
        super(Siamese, self).__init__()
        self.feature_extractor = CNN()

    def forward_pass(self, x1, x2):
        embed1 = self.feature_extractor(x1)
        embed2 = self.feature_extractor(x2)
        embed1 = F.normalize(embed1, p=2, dim=1)
        embed2 = F.normalize(embed2, p=2, dim=1)
        return embed1, embed2
    
siamese_model = Siamese().to(device)

In [61]:
def contrastive_loss(embed1, embed2, label, margin=0.275):
    cosine_sim = F.cosine_similarity(embed1, embed2, dim=1)
    cosine_distance = 1 - cosine_sim

    loss_similar = label * torch.pow(cosine_distance, 2)
    loss_dissimilar = (1-label) * torch.pow(torch.clamp(margin - cosine_distance, min=0.0), 2)

    loss = torch.mean(loss_similar + loss_dissimilar)
    return loss

In [62]:
optimizer = optim.Adam(siamese_model.parameters(), lr=0.0001)

epochs = 10
margin = 0.275

for epoch in range(epochs):
    siamese_model.train()
    running_loss = 0.0
    total_samples = 0
    correct = 0

    for sample1, sample2, pair_label in tqdm(siamese_train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        sample1 = sample1.to(device)
        sample2 = sample2.to(device)
        pair_label = pair_label.to(device)

        optimizer.zero_grad()

        embed1, embed2 = siamese_model.forward_pass(sample1, sample2)

        cosine_sim = F.cosine_similarity(embed1, embed2, dim=1)
        cosine_distance = 1 - cosine_sim
        
        loss = contrastive_loss(embed1, embed2, pair_label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * sample1.size(0)

        preds = (cosine_distance < margin).float()
        correct += (preds == pair_label).sum().item()
        total_samples += sample1.size(0)

    avg_loss = running_loss / total_samples
    accuracy = correct / total_samples
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.4f}, Training Accuracy = {accuracy:.4f}")

Epoch 1/10:   0%|          | 0/668 [00:00<?, ?it/s]

Epoch 1/10:   0%|          | 3/668 [00:25<1:33:51,  8.47s/it]


KeyboardInterrupt: 